In [1]:
# Install dependencies
#%pip install vectorbt
#%pip install "dask[distributed]" --upgrade
# %pip install dask
# Create local directories
#!mkdir -p $HOME/kohv04/lib
#!mkdir -p $HOME/kohv04/lib/python3.10/site-packages

# Download and extract libta-lib to local directory
#!curl -L https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2 | tar xj -C $HOME/kohv04/lib --strip-components=1

# Download ta-lib Python package
#!wget https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2

# Extract ta-lib to local site-packages
#!tar -xjf ta-lib-0.4.19-py310hde88566_4.tar.bz2 -C $HOME/kohv04/lib/python3.10/site-packages --strip-components=2 lib/python3.10/site-packages/talib

# Clean up
#!rm ta-lib-0.4.19-py310hde88566_4.tar.bz2

In [ ]:
# Setting LD_LIBRARY_PATH for libta-lib (required for talib)
import os
os.environ['LD_LIBRARY_PATH'] = f"{os.environ.get('HOME')}/kohv04/lib:{os.environ.get('LD_LIBRARY_PATH', '')}"

# Appending talib path to sys.path (instead of modifying PYTHONPATH)
import sys
sys.path.append('/home/jupyter-kohv04@vse.cz/kohv04/lib/python3.10/site-packages')

from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
import os
import dask.dataframe as dd
import pandas as pd
import vectorbt as vbt
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
try:
    import talib
    print(f"TA-Lib version: {talib.__version__}")
except ImportError as e:
    print(f"Failed to import TA-Lib: {e}")
import pyarrow.parquet as pq
from pathlib import Path
import numpy as np
import glob
import dask
import pyarrow as pa
import json
from dataclasses import dataclass, field
from typing import Dict, Any, List, Tuple, Optional

TA-Lib version: 0.4.19


In [ ]:
# Dask cluster
ProgressBar().register()
temp_dir = "/tmp/dask-spill"
os.makedirs(temp_dir, exist_ok=True)

cluster = LocalCluster(
    n_workers=32,
    threads_per_worker=1,
    memory_limit="10GB",
    processes=True,
    local_directory=temp_dir
)
client = Client(cluster)
print("Dask Dashboard:", client.dashboard_link)

dask.config.set({
    "temporary-directory": temp_dir,
    "distributed.worker.memory.spill": 0.7,
    "distributed.worker.memory.target": 0.6,
    "distributed.worker.memory.pause": 0.85,
    "distributed.worker.memory.terminate": 0.95,
})


# --- Configuration Constants ---

def get_config() -> Dict[str, Any]:
    """
    Encapsulates all configuration constants in a single dictionary for easy management.
    """
    base_dir = "/home/jupyter-kohv04@vse.cz/kohv04/backtesting_final/"
    
    return {
        # --- File Paths ---
        "base_dir": base_dir,
        "metadata_file": f"{base_dir}/metadata/nasdaq100_ticker_dataset.json",

        # --- Date Ranges ---
        "start_buffer": pd.to_datetime("2024-08-01"),
        "start_date": pd.to_datetime("2024-08-15"),
        "opt_end_date": pd.to_datetime("2024-11-14"),
        "sim_start_date": pd.to_datetime("2024-11-15"),
        "end_date": pd.to_datetime("2025-02-14"),

        # --- Trading Hours ---
        "premarket_start": pd.to_datetime("08:30:00").time(),
        "trading_start": pd.to_datetime("09:30:00").time(),
        "trading_end": pd.to_datetime("16:00:00").time(),

        # --- Strategy Parameters ---
        "baseline": {
            "delta_breakout_sl": 0.01,
            "delta_breakout_tp": 0.01,
            "tau_mom": 3,
            "delta_mom_trail": 0.01,
            "tau_bb": 20,
            "gamma_bb": 2,
        },
        "volume_enhanced": {
            "phi_va": 0.7,
            "kappa_surge": 1.5,
            "tau_vol_avg": 14,
            "tau_rsi": 14,
            "theta_rsi_buy": 50,
            "theta_rsi_sell": 50,
            "alpha_vol_breakout_sl": 1,
            "alpha_vol_breakout_tp": 1,
            "tau_obv": 14,
            "theta_obv": 0.2,
            "epsilon_vwap_slope": 0,
            "alpha_vwap_sl": 1,
            "alpha_vwap_tp": 1,
            "kappa_vol_mom": 1.5,
            "tau_adx": 14,
            "theta_adx": 25,
            "alpha_vol_mom_sl": 1,
            "alpha_vol_mom_tp": 1,
            "delta_vol_mom_trail": 0.01,
        },
        "deep_learning": {
            "beta_pred": 0.8,
            "kappa_dl": 2.7,
            "phi_vah": 0.7,
            "phi_val": 0.7,
            "tau_mom_dl": 5,
            "delta_vwap_buy": 0.01,
            "delta_vwap_sell": 0.01,
            "tau_vwap_trend": 5,
            "delta_mom_buy": 0.005,
            "delta_mom_sell": 0.005,
            "delta_dl_sl": 0.01,
            "delta_dl_tp": 0.01,
        }
    }

CONFIG = get_config()

Dask Dashboard: http://127.0.0.1:8787/status


# Optimized

In [ ]:
# --- Helper Functions ---

def _calculate_daily_profile_metrics(daily_profile: pd.Series) -> pd.Series:
    """
    Calculates POC, VAH, and VAL for a single day's volume profile.
    """
    daily_profile = daily_profile.dropna()
    if daily_profile.empty:
        return pd.Series({'poc': np.nan, 'vah': np.nan, 'val': np.nan})

    poc = daily_profile.idxmax()
    total_volume = daily_profile.sum()
    target_volume = total_volume * CONFIG['volume_enhanced']['phi_va']

    sorted_volumes = daily_profile.sort_values(ascending=False)
    
    cum_volume = 0
    value_area_prices = []
    for price, volume in sorted_volumes.items():
        cum_volume += volume
        value_area_prices.append(price)
        if cum_volume >= target_volume:
            break

    vah = max(value_area_prices) if value_area_prices else np.nan
    val = min(value_area_prices) if value_area_prices else np.nan
    
    return pd.Series({'poc': poc, 'vah': vah, 'val': val})

def compute_daily_volume_profile(df: pd.DataFrame, volume_col: str) -> pd.DataFrame:
    """
    Robust computation of Volume Profile (VAH, VAL, POC) for all dates in the DataFrame.
    """
    print(f"Computing daily volume profiles (VAH, VAL, POC) using '{volume_col}'...")
    trading_df = df[df['timestamp'].dt.time.between(CONFIG['trading_start'], CONFIG['trading_end'])].copy()
    if trading_df.empty or volume_col not in trading_df or trading_df[volume_col].isnull().all():
        return pd.DataFrame(columns=['date', 'vah', 'val', 'poc'])

    trading_df['price_bin'] = np.floor(trading_df['close'] * 100) / 100
    
    volume_profiles = trading_df.groupby([trading_df['timestamp'].dt.date, 'price_bin'])[volume_col].sum()
    
    metrics_list = []
    if not volume_profiles.empty:
        for date, group_profile in volume_profiles.groupby(level=0):
            single_day_profile = group_profile.reset_index(level=0, drop=True)
            metrics = _calculate_daily_profile_metrics(single_day_profile)
            metrics['date'] = date
            metrics_list.append(metrics)

    if not metrics_list:
        return pd.DataFrame(columns=['date', 'vah', 'val', 'poc'])
        
    daily_metrics_df = pd.DataFrame(metrics_list)
    return daily_metrics_df[['date', 'poc', 'vah', 'val']]

def compute_daily_vwap(df: pd.DataFrame, volume_col: str) -> pd.Series:
    """
    Vectorized computation of daily VWAP. Can specify which volume column to use.
    """
    print(f"Computing daily VWAP using '{volume_col}'...")
    df_copy = df.copy()
    if volume_col not in df_copy or df_copy[volume_col].isnull().all():
        return pd.Series(index=df.index, dtype=float)

    df_copy['typical_price'] = (df_copy['high'] + df_copy['low'] + df_copy['close']) / 3
    df_copy['price_volume'] = df_copy['typical_price'] * df_copy[volume_col]
    
    date_group = df_copy['timestamp'].dt.date
    cum_price_volume = df_copy.groupby(date_group)['price_volume'].cumsum()
    cum_volume = df_copy.groupby(date_group)[volume_col].cumsum()
    
    vwap = cum_price_volume / cum_volume
    return vwap.replace([np.inf, -np.inf], np.nan)

def load_data_for_ticker(ticker: str) -> Tuple[Optional[pd.DataFrame], Optional[pd.DataFrame], Optional[pd.DataFrame]]:
    """Loads optimization_tft, simulation_tft, and buffer data for a given ticker."""
    print(f"Loading data for ticker: {ticker}")
    ticker_dir = f"{CONFIG['base_dir']}/ticker={ticker}_standardized"
    opt_input_dir = f"{ticker_dir}/optimization_tft"
    sim_input_dir = f"{ticker_dir}/simulation_tft"
    
    if not (os.path.exists(opt_input_dir) and os.path.exists(sim_input_dir)):
        print(f"Warning: TFT directory not found for {ticker}: {opt_input_dir} or {sim_input_dir}")
        return None, None, None

    try:
        df_opt = pd.read_parquet(opt_input_dir)
        df_sim = pd.read_parquet(sim_input_dir)
        
        all_files = glob.glob(f"{ticker_dir}/part.*.parquet")
        df_buffer = pd.concat((pd.read_parquet(f) for f in all_files), ignore_index=True)

        for df_part in [df_opt, df_sim, df_buffer]:
            if 'ticker' not in df_part.columns: df_part['ticker'] = ticker
            df_part['ticker'] = df_part['ticker'].astype(str)
            if not pd.api.types.is_datetime64_any_dtype(df_part['timestamp']):
                df_part['timestamp'] = pd.to_datetime(df_part['timestamp'])
        
        df_buffer = df_buffer[
            (df_buffer['timestamp'] >= CONFIG['start_buffer']) & 
            (df_buffer['timestamp'] < CONFIG['start_date'])
        ].sort_values('timestamp').reset_index(drop=True)

        return df_opt.sort_values('timestamp'), df_sim.sort_values('timestamp'), df_buffer
    except Exception as e:
        print(f"Error loading data for {ticker}: {e}")
        return None, None, None


def calculate_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """
    Orchestrates the calculation of all indicators for a given DataFrame.
    """
    df = df.copy()
    
    print("Computing shared and standard indicators...")
    df['atr'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=CONFIG['volume_enhanced']['tau_adx'])
    df['atr'] = df['atr'].bfill().ffill()
    
    vp_df_actual = compute_daily_volume_profile(df, volume_col='volume')
    if not vp_df_actual.empty:
        vp_df_actual['date'] = pd.to_datetime(vp_df_actual['date'])
        df['date_dt'] = df['timestamp'].dt.normalize()
        df = pd.merge(df, vp_df_actual.add_suffix('_actual'), left_on='date_dt', right_on='date_actual', how='left')
        df.drop(columns=['date_dt', 'date_actual'], inplace=True, errors='ignore')
    else:
        df['vah_actual'] = np.nan
        df['val_actual'] = np.nan
        df['poc_actual'] = np.nan
    df[['vah_actual', 'val_actual', 'poc_actual']] = df[['vah_actual', 'val_actual', 'poc_actual']].ffill().bfill()
    
    df['vwap_actual'] = compute_daily_vwap(df, volume_col='volume')
    
    df = calculate_baseline_indicators(df)
    df = calculate_volume_enhanced_indicators(df)
    df = calculate_deep_learning_indicators(df)
    
    return df

def calculate_baseline_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """Calculates Baseline Strategy indicators."""
    print("Computing Baseline strategy indicators...")
    params = CONFIG['baseline']
    df['breakout_buy'] = df['close'] > df['prev_session_high']
    df['breakout_sell'] = df['close'] < df['prev_session_low']
    df['breakout_stop_loss'] = df['close'] * (1 - params['delta_breakout_sl'])
    df['breakout_take_profit'] = df['close'] * (1 + params['delta_breakout_tp'])
    
    df['momentum_buy'] = (df['close'] > df['open']).rolling(params['tau_mom']).sum() >= params['tau_mom']
    df['momentum_sell'] = (df['close'] < df['open']).rolling(params['tau_mom']).sum() >= params['tau_mom']
    df['momentum_trailing_stop'] = df['high'] * (1 - params['delta_mom_trail'])
    
    bbands = vbt.BBANDS.run(df['close'], window=params['tau_bb'], alpha=params['gamma_bb'])
    df['bb_upper'] = bbands.upper
    df['bb_middle'] = bbands.middle
    df['bb_lower'] = bbands.lower
    df['bb_buy'] = df['close'] < df['bb_lower']
    df['bb_sell'] = df['close'] > df['bb_upper']
    return df

def calculate_volume_enhanced_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """Calculates Volume-Enhanced Strategy indicators based on actual 'volume'."""
    print("Computing Volume-Enhanced strategy indicators...")
    params = CONFIG['volume_enhanced']
    df['volume_avg'] = df['volume'].rolling(window=params['tau_vol_avg']).mean().ffill().bfill()
    df['volume_surge'] = df['volume'] > (params['kappa_surge'] * df['volume_avg'])
    df['relative_volume'] = df['volume'] / df['volume_avg'].replace(0, np.nan)
    
    df['rsi'] = vbt.RSI.run(df['close'], window=params['tau_rsi']).rsi
    df['obv'] = vbt.OBV.run(df['close'], df['volume']).obv
    df['obv_threshold'] = df['obv'].rolling(params['tau_obv']).quantile(params['theta_obv'])
    df['adx'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=params['tau_adx'])
    df['vwap_actual_slope'] = df['vwap_actual'].diff()
    
    df['volume_breakout_buy'] = (df['close'] > df['vah_actual']) & df['volume_surge'] & (df['rsi'] > params['theta_rsi_buy'])
    df['volume_breakout_sell'] = (df['close'] < df['val_actual']) & df['volume_surge'] & (df['rsi'] < params['theta_rsi_sell'])
    df['volume_breakout_stop_loss'] = df['close'] - (params['alpha_vol_breakout_sl'] * df['atr'])
    df['volume_breakout_take_profit'] = df['close'] + (params['alpha_vol_breakout_tp'] * df['atr'])
    
    df['vwap_buy'] = (df['close'] < df['vwap_actual']) & (df['obv'] < df['obv_threshold']) & (df['vwap_actual_slope'] >= params['epsilon_vwap_slope'])
    df['vwap_sell'] = (df['close'] > df['vwap_actual']) & (df['obv'] > df['obv_threshold']) & (df['vwap_actual_slope'] <= -params['epsilon_vwap_slope'])
    df['vwap_stop_loss'] = df['close'] - (params['alpha_vwap_sl'] * df['atr'])
    df['vwap_take_profit'] = df['close'] + (params['alpha_vwap_tp'] * df['atr'])
    
    df['volume_momentum_buy'] = (df['relative_volume'] > params['kappa_vol_mom']) & (df['close'] > df['prev_session_high']) & (df['adx'] > params['theta_adx'])
    df['volume_momentum_sell'] = (df['relative_volume'] > params['kappa_vol_mom']) & (df['close'] < df['prev_session_low']) & (df['adx'] > params['theta_adx'])
    df['volume_momentum_stop_loss'] = df['close'] - (params['alpha_vol_mom_sl'] * df['atr'])
    df['volume_momentum_take_profit'] = df['close'] + (params['alpha_vol_mom_tp'] * df['atr'])
    df['volume_momentum_trailing_stop'] = df['high'] * (1 - params['delta_vol_mom_trail'])
    
    return df

def calculate_deep_learning_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates Deep Learning-Enhanced Strategy indicators, using 'pred_volume_15_tft'.
    """
    print("Computing Deep Learning-Enhanced strategy indicators...")
    params = CONFIG['deep_learning']
    if 'pred_volume_15_tft' not in df.columns:
        df['pred_volume_15_tft'] = np.nan
    
    df['pred_volume_tft_15_scaled'] = df['pred_volume_15_tft'] * params['beta_pred']
    
    vp_df_dl = compute_daily_volume_profile(df, volume_col='pred_volume_tft_15_scaled')
    if not vp_df_dl.empty:
        vp_df_dl['date'] = pd.to_datetime(vp_df_dl['date'])
        df['date_dt'] = df['timestamp'].dt.normalize()
        df = pd.merge(df, vp_df_dl.add_suffix('_dl'), left_on='date_dt', right_on='date_dl', how='left')
        df.drop(columns=['date_dt', 'date_dl'], inplace=True, errors='ignore')
    else:
        df['vah_dl'] = np.nan
        df['val_dl'] = np.nan
        df['poc_dl'] = np.nan
    df[['vah_dl', 'val_dl', 'poc_dl']] = df[['vah_dl', 'val_dl', 'poc_dl']].ffill().bfill()
    
    df['vwap_dl'] = compute_daily_vwap(df, volume_col='pred_volume_tft_15_scaled')
    
    df['dl_volume_breakout_buy'] = (df['close'] > params['phi_vah'] * df['vah_dl']) & (df['pred_volume_tft_15_scaled'] > params['kappa_dl'] * df['volume']) & (df['close'] > df['close'].shift(params['tau_mom_dl']))
    df['dl_volume_breakout_sell'] = (df['close'] < params['phi_val'] * df['val_dl']) & (df['pred_volume_tft_15_scaled'] > params['kappa_dl'] * df['volume']) & (df['close'] < df['close'].shift(params['tau_mom_dl']))
    df['dl_volume_breakout_stop_loss'] = df['close'] * (1 - params['delta_dl_sl'])
    df['dl_volume_breakout_take_profit'] = df['close'] * (1 + params['delta_dl_tp'])

    df['dl_vwap_buy'] = (df['close'] < (1 - params['delta_vwap_buy']) * df['vwap_dl']) & (df['pred_volume_tft_15_scaled'] > params['kappa_dl'] * df['volume']) & (df['vwap_dl'] < df['vwap_dl'].shift(params['tau_vwap_trend']))
    df['dl_vwap_sell'] = (df['close'] > (1 + params['delta_vwap_sell']) * df['vwap_dl']) & (df['pred_volume_tft_15_scaled'] > params['kappa_dl'] * df['volume']) & (df['vwap_dl'] > df['vwap_dl'].shift(params['tau_vwap_trend']))
    df['dl_vwap_stop_loss'] = df['close'] * (1 - params['delta_dl_sl'])
    df['dl_vwap_take_profit'] = df['close'] * (1 + params['delta_dl_tp'])
    
    df['dl_volume_momentum_buy'] = (df['close'] > (1 + params['delta_mom_buy']) * df['prev_session_high']) & (df['pred_volume_tft_15_scaled'] > params['kappa_dl'] * df['volume'])
    df['dl_volume_momentum_sell'] = (df['close'] < (1 - params['delta_mom_sell']) * df['prev_session_low']) & (df['pred_volume_tft_15_scaled'] > params['kappa_dl'] * df['volume'])
    df['dl_volume_momentum_stop_loss'] = df['close'] * (1 - params['delta_dl_sl'])
    df['dl_volume_momentum_take_profit'] = df['close'] * (1 + params['delta_dl_tp'])
    
    return df

def process_ticker(ticker: str) -> Dict[str, Any]:
    """Main processing pipeline for a single ticker."""
    print(f"\n--- Processing ticker: {ticker} ---")
    
    df_opt, df_sim, df_buffer = load_data_for_ticker(ticker)
    if df_opt is None:
        return {'ticker': ticker, 'error': f"TFT data loading failed for {ticker}"}

    required_cols = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'prev_session_high', 'prev_session_low', 'pred_volume_15_tft']
    if any(col not in df_opt.columns for col in required_cols) or \
       any(col not in df_sim.columns for col in required_cols):
        missing = [c for c in required_cols if c not in df_opt.columns or c not in df_sim.columns]
        return {'ticker': ticker, 'error': f"Missing required columns for {ticker}: {missing}"}

    df_full = pd.concat([df_buffer, df_opt, df_sim], ignore_index=True).sort_values('timestamp').reset_index(drop=True)
    
    df_indicators = calculate_indicators(df_full)

    # --- Split and Save ---
    time_filter = df_indicators['timestamp'].dt.time.between(CONFIG['premarket_start'], CONFIG['trading_end'])
    
    df_opt_final = df_indicators[(df_indicators['timestamp'] >= CONFIG['start_date']) & (df_indicators['timestamp'] <= CONFIG['opt_end_date']) & time_filter].copy()
    df_sim_final = df_indicators[(df_indicators['timestamp'] >= CONFIG['sim_start_date']) & (df_indicators['timestamp'] <= CONFIG['end_date']) & time_filter].copy()
    
    original_cols = df_opt.columns.tolist()
    baseline_cols = original_cols + ['atr', 'vah_actual', 'val_actual', 'poc_actual', 'vwap_actual', 'breakout_buy', 'breakout_sell', 'breakout_stop_loss', 'breakout_take_profit', 'momentum_buy', 'momentum_sell', 'momentum_trailing_stop', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_buy', 'bb_sell']
    volume_enhanced_cols = original_cols + ['atr', 'vah_actual', 'val_actual', 'poc_actual', 'vwap_actual', 'volume_avg', 'volume_surge', 'relative_volume', 'rsi', 'obv', 'obv_threshold', 'adx', 'vwap_actual_slope', 'volume_breakout_buy', 'volume_breakout_sell', 'volume_breakout_stop_loss', 'volume_breakout_take_profit', 'vwap_buy', 'vwap_sell', 'vwap_stop_loss', 'vwap_take_profit', 'volume_momentum_buy', 'volume_momentum_sell', 'volume_momentum_stop_loss', 'volume_momentum_take_profit', 'volume_momentum_trailing_stop']
    deep_learning_cols = volume_enhanced_cols + ['pred_volume_15_tft', 'pred_volume_tft_15_scaled', 'vah_dl', 'val_dl', 'poc_dl', 'vwap_dl', 'dl_volume_breakout_buy', 'dl_volume_breakout_sell', 'dl_volume_breakout_stop_loss', 'dl_volume_breakout_take_profit', 'dl_vwap_buy', 'dl_vwap_sell', 'dl_vwap_stop_loss', 'dl_vwap_take_profit', 'dl_volume_momentum_buy', 'dl_volume_momentum_sell', 'dl_volume_momentum_stop_loss', 'dl_volume_momentum_take_profit']

    strategy_data_map = {
        'Baseline': (baseline_cols),
        'Volume_Enhanced': (volume_enhanced_cols),
        'Deep_Learning_Enhanced': (deep_learning_cols),
    }

    print(f"Saving updated Parquet files for {ticker}...")
    ticker_dir = f"{CONFIG['base_dir']}/ticker={ticker}_standardized"
    for strategy_name, cols_to_save in strategy_data_map.items():
        for period, df_period in [('optimization', df_opt_final), ('simulation', df_sim_final)]:
            output_dir = f"{ticker_dir}/{period}_indicators_{strategy_name}"
            os.makedirs(output_dir, exist_ok=True)
            if not df_period.empty:
                df_to_save = pd.DataFrame()
                for col in list(dict.fromkeys(cols_to_save)): 
                    if col in df_period.columns:
                        df_to_save[col] = df_period[col]
                    else:
                        df_to_save[col] = np.nan
                
                df_to_save['date'] = df_to_save['timestamp'].dt.date
                for date_val, group in df_to_save.groupby('date'):
                    output_path = f"{output_dir}/part.{date_val.strftime('%Y-%m-%d')}.parquet"
                    group.drop(columns=['date']).to_parquet(output_path, index=False)
    
    print(f"Finished processing ticker: {ticker}")
    return {'ticker': ticker, 'error': None}

In [ ]:
def main():
    """Main execution function."""
    print(f"Loading tickers from {CONFIG['metadata_file']}")
    try:
        with open(CONFIG['metadata_file'], 'r') as f:
            tickers = [item['Ticker'] for item in json.load(f)]
        # demonstration - ticker [:1] 
        tickers_to_process = tickers
        print(f"Loaded {len(tickers)} tickers. Processing: {tickers_to_process}")
    except FileNotFoundError:
        print(f"Error: Metadata file not found at {CONFIG['metadata_file']}")
        return

    print("Starting synchronous processing of tickers...")
    results = [process_ticker(ticker) for ticker in tickers_to_process]
    
    print("\n--- Processing complete. Summary of results: ---")
    for result in results:
        if result.get('error'):
            print(f"Ticker {result['ticker']}: Failed with error - {result['error']}")
        else:
            print(f"Ticker {result['ticker']}: Processed successfully")

if __name__ == "__main__":
    main()

Loading tickers from /home/jupyter-kohv04@vse.cz/kohv04/backtesting_final//metadata/nasdaq100_ticker_dataset.json
Loaded 101 tickers. Processing: ['AAPL', 'ABNB', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'ANSS', 'APP', 'ARM', 'ASML', 'AVGO', 'AXON', 'AZN', 'BIIB', 'BKNG', 'BKR', 'CCEP', 'CDNS', 'CDW', 'CEG', 'CHTR', 'CMCSA', 'COST', 'CPRT', 'CRWD', 'CSCO', 'CSGP', 'CSX', 'CTAS', 'CTSH', 'DASH', 'DDOG', 'DXCM', 'EA', 'EXC', 'FANG', 'FAST', 'FTNT', 'GEHC', 'GFS', 'GILD', 'GOOG', 'GOOGL', 'HON', 'IDXX', 'INTC', 'INTU', 'ISRG', 'KDP', 'KHC', 'KLAC', 'LIN', 'LRCX', 'LULU', 'MAR', 'MCHP', 'MDB', 'MDLZ', 'MELI', 'META', 'MNST', 'MRVL', 'MSFT', 'MSTR', 'MU', 'NFLX', 'NVDA', 'NXPI', 'ODFL', 'ON', 'ORLY', 'PANW', 'PAYX', 'PCAR', 'PDD', 'PEP', 'PLTR', 'PYPL', 'QCOM', 'REGN', 'ROP', 'ROST', 'SBUX', 'SNPS', 'TEAM', 'TMUS', 'TSLA', 'TTD', 'TTWO', 'TXN', 'VRSK', 'VRTX', 'WBD', 'WDAY', 'XEL', 'ZS']
Starting synchronous processing of tickers...

--- Processing ticker: AAPL --